# 1 - Data acquisition

## Description of the tables

### enrollments.csv:
It's basically a table collecting the data of all Data Analyst Nanodegree students who complete their first project (i.e. enrolling) and those who do not. 
Each student is identified by an ACCOUNT_KEY and has a current STATUS, which could be either 'current' or 'canceled' (in the second case, the property IS_CANCELED confirms).
Each student enrolled on a JOIN_DATE and may have canceled on a CANCEL_DATE (blank if the student has not yet canceled), therefore it has been enrolled for a DAYS_TO_CANCEL days (difference between the join_date and the cancel_date, or blank if the student has not yet canceled). Each student may have an IS_UDACITY type of account.
So, summarizing:
- ACCOUNT_KEY: a unique identifier for the account of the student who enrolled
- STATUS: the enrollment status of the student (at the time the data was collected), it can be 'CANCELED' or 'CURRENT'
- JOIN_DATE: the date the student enrolled
- CANCEL_DATE: the date the student canceled, or blank if the student has not yet canceled
- DAYS_TO_CANCEL: the number of days between JOIN_DATE and CANCEL_DATE, or blank if the student has not yet canceled
- IS_UDACITY: true if the account is a 'Udacity test' account, False otherwise
- IS_CANCELED: True if the student has canceled his enrollment at the time the data was collected, False otherwise


### daily_engagement.csv:
This table summarizes the engagement of each student, identified by the ACCT account number. Data was collected on day UTC_DATE, therefore the rest of the content refers to this date. Specifically, at the same day, the student has visited NUM_COURSES_VISITED amount of courses, spent TOTAL_MINUTES_VISITED minutes visiting courses, completed LESSONS_COMPLETED courses and completed PROJECT_COMPLETED projects.
Summarizing:
- ACCT: a unique identifier for the account of the student
- UTC_DATE: the date for which the data was collected
- NUM_COURSES_VISITED: the total number of courses of the Data Analyst Nanodegree courses the student visited on the current day
- TOTAL_MINUTES_VISITED: the total number of courses the student spent taking Data Analysis Nanodegree courses today
- LESSONS_COMPLETED: total number of lessons within Data Analyst Nanodegree courses on this day
- PROJECTS_COMPLETED: total number of Data Analyst Nanodegree projects the student completed on this day


### project_submissions.csv:
This table contains, for each students identified uniquely by an ACCOUNT_KEY, the state of his/her projects on the Data Analyst Nanodegree. Each student submit a project belongin to the LESSON_KEY course on the CREATION_DATE: the project is being evaluated until the COMPLETION_DATE. During the process, to the project a PROCESSING_STATE is assigned. Once the project has been evaluated, an ASSIGNED_RATING value is assigned to the project.

Summarizing:
- CREATION_DATE: the date the project was submitted
- COMPLETION_DATE: the date the project was evaluated
- ASSIGNED_RATING: the rating given to the project, it can be
                      - blank: the project has not yet been evaluated
                      - INCOMPLETE: the project did not meet the specifications required
                      - PASSED: the project met the specifications required
                      - DISTINCTION: the project exceeded the specifications required
                      - UNGRADED: the submission could not be evaluated (e.g. corrupted file submitted)
- ACCOUNT_KEY: a unique identifier for the account of the student who submitted the project
- LESSON_KEY: a unique identifier for the project submitted
- PROCESSING_STATE: the processing state of the project, it can be
                      - CREATED: the project has been submitted, but not evaluated yet
                      - EVALUATED: the project has been evaluated

In [228]:
# Preparing the environment with the DEPENDECIES:
import unicodecsv as ucsv # CSV file reading package

## Acquire the data using CSV files

### Method 1: the long road

In [229]:
# Import the UNICODECSV package
import unicodecsv as ucsv
# Create an empty list for enrollments
enrollments = []
# Open the content of the file and store it into a Dictionary called READER using the DICTREADER method of the UNICODECSV
f = open('enrollments.csv','rb')
reader = ucsv.DictReader(f)
# Fill the enrollment list with each row of the READER dictionary
for row in reader:
    enrollments.append(row)
# Close the file
f.close()
# Print the first record of the list
enrollments[0]

{'account_key': '448',
 'cancel_date': '2015-01-14',
 'days_to_cancel': '65',
 'is_canceled': 'True',
 'is_udacity': 'True',
 'join_date': '2014-11-10',
 'status': 'canceled'}

### Method 2: quicker functions

In [230]:
# Create an empty list for enrollments
enrollments = []
# Open the file and automatically close it with the WITH OPEN method (everything following command included while the file is opened must be indented)
with open('enrollments.csv','rb') as f:
    # Read the content from the file using the DICTREADER method of the UNICODECSV package
    reader = ucsv.DictReader(f)
    # Convert the dictionary READER into a list using the LIST function (this also initialize the list, so you don't have to create it)
    enrollments = list(reader)

# Print the first record of the list
enrollments[0]

{'account_key': '448',
 'cancel_date': '2015-01-14',
 'days_to_cancel': '65',
 'is_canceled': 'True',
 'is_udacity': 'True',
 'join_date': '2014-11-10',
 'status': 'canceled'}

#### Writing a function to do this:

In [231]:
#######################################################
def read_csv(filename):
# READ_CSV(FILENAME) the function gets a filename - with its full path - as an input and returns the list of the records in the CSV file, already processed as a list of dictionaries (i.e., keys and values)
    with open(filename,'rb') as file:
        dict_of_csv = ucsv.DictReader(file)
        list_of_csv = list(dict_of_csv)
        return list_of_csv
#######################################################

#### Applying the function

In [232]:
daily_engagement = read_csv('daily_engagement.csv')
print(daily_engagement[0])
project_submissions = read_csv('project_submissions.csv')
print(project_submissions[0])

{'projects_completed': '0.0', 'lessons_completed': '0.0', 'utc_date': '2015-01-09', 'num_courses_visited': '1.0', 'acct': '0', 'total_minutes_visited': '11.6793745'}
{'completion_date': '2015-01-16', 'processing_state': 'EVALUATED', 'assigned_rating': 'UNGRADED', 'creation_date': '2015-01-14', 'lesson_key': '3176718735', 'account_key': '256'}


# 2 - Data Cleaning: clean up the tables

## Fixing data types
Checking if all data, even though they're imported as strings, can be converted to the proper data type (INT, BOOL, STR, etc.)

### Defining a few important functions

In [233]:
############################################################################
def parse_date(date):
    if date=='':
        return None
    else:
        return datetime.datetime.strptime(date, "%Y-%m-%d")
############################################################################

In [234]:
############################################################################
def parse_int(integer):
    # Check if the input is not empty
    if integer=='':
        return None
    # Convert the string in integer using the INT method
    else:
        return int(integer)
############################################################################

### Proceeding with the actual clean-up

In [235]:
# For each record in the ENROLLMENTS list
for enrollment in enrollments:
    # Use the PARSE_DATE function to convert each string-date into a datetime-date
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    # Use the PARSE_INT function to convert each string-int into an integer-int
    enrollment['account_key'] = parse_int(enrollment['account_key'])
    enrollment['days_to_cancel'] = parse_int(enrollment['days_to_cancel'])
    # Use the normal boolean functions to convert TRUE-FALSE strings into boolean values
    enrollment['is_canceled'] = (enrollment['is_canceled']=='True')
    enrollment['is_udacity'] = (enrollment['is_udacity']=='True')

In [236]:
# For each record in the DAILY_ENGAGEMENT list
for engagement in daily_engagement:
    # From STRING to INT
    engagement['acct'] = parse_int(engagement['acct'])
    # From STRING to FLOAT
    engagement['total_minutes_visited'] = float(engagement['total_minutes_visited'])
    # From STRING to DATE
    engagement['utc_date'] = parse_date(engagement['utc_date'])
    # From STRING to FLOAT to INT
    engagement['lessons_completed'] = int(float(engagement['lessons_completed']))
    engagement['projects_completed'] = int(float(engagement['projects_completed']))
    engagement['num_courses_visited'] = int(float(engagement['num_courses_visited']))

In [237]:
# Proof-checking with another print
print(type(engagement['lessons_completed']))

<class 'int'>


In [238]:
# For each record in the PROJECT_SUBMISSIONS list
for submission in project_submissions:
    # From STRING to DATE
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])
    # From STRING to INT
    submission['account_key'] = parse_int(submission['account_key'])
    submission['lesson_key'] = parse_int(submission['lesson_key'])

In [239]:
# Proof-checking with another print
print(type(submission['lesson_key']))

<class 'int'>


## Filtering the unique content
E.g. find the number of UNIQUE records, depending on the primary key (in this example, the account key)

### Before investigating the data, let's fix a little thing
Apparently, the same value in ENROLLMENTS.CSV and in PROJECT_SUBMISSIONS.CSV is called ACCOUNT_KEY, while in DAILY_ENGAGEMENT.CSV is called ACCT. Let's call it the same way!

In [240]:
############################################################################
# Function takes the DATA set as input and replaces the given OLD_KEY with the NEW_KEY
# DATA is a list of dictionaries, OLD_KEY and NEW_KEY are strings
def replace_key(data,old_key,new_key):
    for record in data:
        # Assign the old key to the new one
        record[new_key] = record[old_key]
        # Cancel the old one
        del(record[old_key])
############################################################################

In [241]:
replace_key(daily_engagement,'acct','account_key')
print(daily_engagement[0]['account_key'])   

0


### Now let's start counting: first a useful function

In [242]:
############################################################################
def get_uniques(data):
    # Amount of unique values in the list (using the SET function)
    uniques = set()
    for record in data:
        # This property of the sets allows us to count only unique records (because the SET itself only contains unique values)
        uniques.add(record['account_key'])
    return uniques
############################################################################

### Then...

In [243]:
# Length of the ENROLLMENT list
enrollment_num_rows = len(enrollments)
print(enrollment_num_rows)

# Amount of unique students in the ENROLLMENT list (long method)
# enrollment_num_unique_students = 0
# account_keys_list = []
# for enrollment in enrollments:
#     if enrollment['account_key'] not in account_keys_list:
#         account_keys_list.append(enrollment['account_key'])
#         enrollment_num_unique_students += 1

# Faster method (using the function above)
enrollment_unique_students = get_uniques(enrollments)
print(len(enrollment_unique_students))

1640
1302


In [244]:
# Similarly for the DAILY_ENGAGEMENT list
engagement_num_rows = len(daily_engagement)
print(engagement_num_rows)

# Amount of the unique students in the DAILY_ENGAGEMENT list
# engagement_num_unique_students = 0
# acct_list = []
# for engagement in daily_engagement:
#     if engagement['acct'] not in acct_list:
#         acct_list.append(engagement['acct'])
#         engagement_num_unique_students += 1
# print(engagement_num_unique_students)

# Faster method (using the function above)
engagement_unique_students = get_uniques(daily_engagement)
print(len(engagement_unique_students))

136240
1237


In [245]:
# Similarly for the PROJECT_SUBMISSIONS list
submission_num_rows = len(project_submissions)
print(submission_num_rows)

# Amount of the unique students in the DAILY_ENGAGEMENT list
# submission_num_unique_students = 0
# account_keys_list = []
# for submission in project_submissions:
#     if submission['account_key'] not in account_keys_list:
#         account_keys_list.append(submission['account_key'])
#         submission_num_unique_students += 1
# print(submission_num_unique_students)

# Faster method (using the function above)
submission_unique_students = get_uniques(project_submissions)
print(len(submission_unique_students))

3642
743


## Data Consistency

It appears that there are more unique records in the ENROLLMENT table than in the DAILY_ENGAGEMENT table... apparently, some students never actually engaged in any courses.'

In [246]:
# A possible way to find out the account_key in the intersection between the ENROLLMENT table and the DAILY_ENGAGEMENT table...
# intersection_enrollment_engagement = (enrollment_unique_students) ^ (engagement_unique_students)

# Otherwise, just proceeding with a FOR loop, breaking at the first record found, with its whole content:
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in engagement_unique_students:
        print(enrollment)
        break

{'is_udacity': False, 'days_to_cancel': 0, 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'is_canceled': True, 'status': 'canceled', 'account_key': 1219, 'cancel_date': datetime.datetime(2014, 11, 12, 0, 0)}


As you may see, the inconsistency could be explained like this: a student listed in the ENROLLMENTS table needs to be enrolled for *at least* one full day in order to be considered in the DAILY_ENGAGEMENT table.

What if we look for the students with one full day of enrollment (so 'days_to_cancel' >= 0 in the ENROLLMENTS table), but still not present in the DAILY_ENGAGEMENT table?

In [247]:
# What we're looking for is the following: students that are enrolled but not in the DAILY_ENGAGEMENT table, 
# and at the same time they have to be students who have a JOIN_DATE and CANCEL_DATE that are different 
# (i.e. students that actually enrolled and stayed more than one day)
records = []
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in engagement_unique_students and enrollment['join_date']!=enrollment['cancel_date']:
        records.append(enrollment)
print(records)

[{'is_udacity': True, 'days_to_cancel': 59, 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'is_canceled': True, 'status': 'canceled', 'account_key': 1304, 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0)}, {'is_udacity': True, 'days_to_cancel': 99, 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'is_canceled': True, 'status': 'canceled', 'account_key': 1304, 'cancel_date': datetime.datetime(2015, 6, 17, 0, 0)}, {'is_udacity': True, 'days_to_cancel': None, 'join_date': datetime.datetime(2015, 2, 25, 0, 0), 'is_canceled': False, 'status': 'current', 'account_key': 1101, 'cancel_date': None}]
